# 1. Data Generator
- Raw Data를 읽어옴
- 여기서 만들어진 데이터는 모델의 입력으로 들어감

In [1]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import Sequence

In [2]:
class RawForVAEGenerator(Sequence):
    def __init__(self, source, wav_dir, files, sourNum='s1', batch_size=10, shuffle=True):
        self.source = source
        self.wav_dir = wav_dir
        self.files = files
        self.sourNum = sourNum
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
        self.sample_rate = 8000
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.source))
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __audioread__(self, path, offset=0.0, duration=None, sample_rate=16000):
        signal = librosa.load(path, sr=self.sample_rate, mono=False, offset=offset, duration=duration)

        return signal[0]
    
    def __padding__(self, data):
        n_batch = len(data)
        max_len = max([d.shape[0] for d in data])
        extrapadding = int(np.ceil(max_len / self.sample_rate) * self.sample_rate)
        pad = np.zeros((n_batch, extrapadding))
        
        for i in range(n_batch):
            pad[i, :data[i].shape[0]] = data[i]
        
        return np.expand_dims(pad, -1)
        
    def __data_generation__(self, source_list):
        wav_list = []
        for name in source_list:
            name = name.strip('\n')
            
            s_wav_name = self.wav_dir + self.files + '/' + self.sourNum + '/' + name
            
            # ------- AUDIO READ -------
            s_wav = (self.__audioread__(s_wav_name,  offset=0.0, duration=None, sample_rate=self.sample_rate))
            # --------------------------
            
            # ------- PADDING -------
#             pad_len = max(len(samples1),len(samples2))
#             pad_s1 = np.concatenate([s1_wav, np.zeros([pad_len - len(s1_wav)])])
            
#             extrapadding = ceil(len(pad_s1) / sample_rate) * sample_rate - len(pad_s1)
#             pad_s1 = np.concatenate([pad_s1, np.zeros([extrapadding - len(pad_s1)])])
#             pad_s2 = np.concatenate([s2_wav, np.zeros([extrapadding - len(s2_wav)])])
            # -----------------------
            
            wav_list.append(s_wav)
        
        return wav_list, wav_list, source_list
            
    
    def __len__(self):
        return int(np.floor(len(self.source) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        source_list = [self.source[k] for k in indexes]
        
        if self.files is not 'tt':
            sour, labels, _ = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            label_pad = self.__padding__(labels) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, np.concatenate([label_pad, exp], axis=1)
        else:
            sour, labels, name = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, exp, name

## Data를 어떻게 읽는지에 대한 부분

In [3]:
WAV_DIR = './mycode/wsj0_2mix/use_this/'
LIST_DIR = './mycode/wsj0_2mix/use_this/lists/'

In [4]:
# Directory List file create

wav_dir = WAV_DIR
output_lst = LIST_DIR

for folder in ['tr', 'cv', 'tt']:
    wav_files = os.listdir(wav_dir + folder + '/mix')
    output_lst_files = output_lst + folder + '_wav.lst'
    with open(output_lst_files, 'w') as f:
        for file in wav_files:
            f.write(file + "\n")

print("Generate wav file to .lst done!")

Generate wav file to .lst done!


In [5]:
batch_size = 1

train_dataset = 0
valid_dataset = 0
test_dataset = 0

name_list = []
for files in ['tr', 'cv', 'tt']:
    # --- Lead lst file ---""
    output_lst_files = LIST_DIR + files + '_wav.lst'
    fid = open(output_lst_files, 'r')
    lines = fid.readlines()
    fid.close()
    # ---------------------
    
    if files == 'tr':
        train_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    elif files == 'cv':
        valid_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    else:
        test_batch = 1
        test_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', test_batch)

# 2. Building VQ-VAE model with Gumbel Softmax

In [6]:
import threading
from scipy.io.wavfile import write as wav_write
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras import backend as Kb
import numpy as np
import pandas as pd
from importlib import reload
import time
from tensorflow.keras.models import Model, Sequential, load_model

In [7]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [8]:
mkdir_p('./CKPT/') # model check point 폴더 만드는 코드
filepath = "./CKPT/CKP_ep_{epoch:d}__loss_{val_loss:.5f}_.h5"

In [9]:
initial_learning_rate = 0.001

# learning rate를 점점 줄이는 부분
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

# validation loss에 대해서 좋은 것만 저장됨
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min'
)

# early stop 하는 부분인데, validation loss에 대해서 제일 좋은 모델이 저장됨
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True
)

In [147]:
class GumbelSoftmax(layers.Layer):
    def __init__(self, temperature=0.5, hard=False, name = 'gumbel_softmax',**kwargs):
        super(GumbelSoftmax, self).__init__(name=name, **kwargs)
        
        self.temperature = temperature
        self.hard = hard
    
    def sample_gumbel(self, shape, eps=1e-20): 
        """Sample from Gumbel(0, 1)"""
        U = tf.random.uniform(shape,minval=0,maxval=1)
        
        return -tf.math.log(-tf.math.log(U + eps) + eps)

    def gumbel_softmax_sample(self, logits, temperature): 
        """ Draw a sample from the Gumbel-Softmax distribution"""
        y = logits + self.sample_gumbel(tf.shape(logits))
        
        return tf.nn.softmax(y / temperature)

    def call(self, inputs):
        y = self.gumbel_softmax_sample(inputs, self.temperature)
        
        if self.hard:
            y_hard = tf.cast(tf.equal(y, tf.math.reduce_max(y, 2, keepdims=True)), y.dtype)
            y = tf.stop_gradient(y_hard - y) + y
        
        return y


class Encoder(layers.Layer):
    def __init__(self, latent_dim, name = 'encoder',**kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.conv1d_1 = layers.Conv1D(filters=32, kernel_size=4, strides=2, activation='relu', padding='same')
        self.conv1d_2 = layers.Conv1D(filters=128, kernel_size=4, strides=2, activation='relu', padding='same')
        self.conv1d_3 = layers.Conv1D(filters=128, kernel_size=4, strides=2, activation='relu', padding='same')
        self.conv1d_4 = layers.Conv1D(filters=256, kernel_size=4, strides=2, activation='relu', padding='same')
        self.conv1d_5 = layers.Conv1D(filters=512, kernel_size=4, strides=2, activation='relu', padding='same')
        self.logit = layers.Conv1D(filters=latent_dim, kernel_size=1, strides=1, activation=None, padding='valid')
    
    def call(self, inputs):
        x = self.conv1d_1(inputs)
        x = self.conv1d_2(x)
        x = self.conv1d_3(x)
        x = self.conv1d_4(x)
        x = self.conv1d_5(x)
        logit = self.logit(x)
        
        return logit


class Decoder(layers.Layer):
    def __init__(self, latent_dim, name = 'decoder',**kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.trans_conv1d_1 = layers.Conv1DTranspose(filters=512, kernel_size=1, strides=1, activation='relu', padding='same')
        self.trans_conv1d_2 = layers.Conv1DTranspose(filters=256, kernel_size=4, strides=2, activation='relu', padding='same')
        self.trans_conv1d_3 = layers.Conv1DTranspose(filters=128, kernel_size=4, strides=2, activation='relu', padding='same')
        self.trans_conv1d_4 = layers.Conv1DTranspose(filters=128, kernel_size=4, strides=2, activation='relu', padding='same')
        self.trans_conv1d_5 = layers.Conv1DTranspose(filters=32, kernel_size=4, strides=2, activation='relu', padding='same')
        self.logit = layers.Conv1DTranspose(filters=1, kernel_size=4, strides=2, activation=None, padding='same')
    
    def call(self, inputs):
        x = self.trans_conv1d_1(inputs)
        x = self.trans_conv1d_2(x)
        x = self.trans_conv1d_3(x)
        x = self.trans_conv1d_4(x)
        x = self.trans_conv1d_5(x)
        logit = self.logit(x)
        
        return logit

In [148]:
# Custom Metric Si-sdr

class SiSdr(keras.metrics.Metric):
    def __init__(self, name="Si-sdr", **kwargs):
        super(SiSdr, self).__init__(name=name, **kwargs)
        self.sdr = self.add_weight(name="sdr", initializer="zeros")
        self.count = self.add_weight(name="cnt", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        ori_length = tf.shape(y_true)[1]
        
        # Label & Length divide
        labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
        lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]
        
        # Check sequence length
        batch_size = tf.shape(labels)[0]
        label_size = tf.shape(labels)[1]
        pred_size = tf.shape(y_pred)[1]
        feature_size = tf.shape(labels)[-1]
        
        # Change sequence length
        if label_size < pred_size:
            y_pred = tf.slice(y_pred, [0, 0, 0], [-1, label_size, -1])
        elif label_size > pred_size:
            labels = tf.slice(labels, [0, 0, 0], [-1, pred_size, -1])

        # SI-SDR
        target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
        noise = y_pred - target
        values = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.sdr.assign_add(tf.reduce_sum(values))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.sdr / self.count

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.sdr.assign(0.0)
        self.count.assign(0.0)

In [149]:
# Custom loss

# Custom mse
def custom_mse(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 129]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    loss = tf.reduce_sum(tf.pow(y_pred - labels, 2), axis=[1, 2])
    loss = tf.reduce_mean(loss)

    return loss


# Custom si-sdr loss
def custom_sisdr_loss(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
    noise = y_pred - target
    si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
    si_sdr = tf.reduce_mean(si_sdr) * -1

    return si_sdr

In [166]:
class Vq_vae(keras.Model):
    def __init__(self, latent_dim, gumbel_hard=False, name='vqvae', **kwargs):
        super(Vq_vae, self).__init__(name=name, **kwargs)
        
        self.latent_dim = latent_dim
        self.softmax = layers.Softmax(-1)
        
        self.encoder = Encoder(latent_dim)
        self.embeddings = layers.Embedding(latent_dim, latent_dim)
        self.decoder = Decoder(latent_dim)
        self.gumbel = GumbelSoftmax(hard=gumbel_hard)
        
    def call(self, inputs, load=False):
        if load:
            inputs = layers.Input(shape=(None, 1))
        
        
        encode = self.encoder(inputs)
        gumbel = self.gumbel(encode)
        decode = self.decoder(gumbel)
        
        # ------------------ KL loss ------------------
        qy = self.softmax(encode)
        log_qy = tf.math.log(qy + 1e-10)
        log_uniform = qy * (log_qy - tf.math.log(1.0 / self.latent_dim))
        kl_loss = tf.reduce_sum(log_uniform, axis=[1, 2])
        kl_loss = tf.reduce_mean(kl_loss) * 0.1
        # ---------------------------------------------
        
        self.add_loss(kl_loss)
        
        return decode

# 이렇게 GradientTape 를 사용해서 프로그램 해도 됨

In [173]:
tf.random.set_seed(42)

latent_size = 1024
epochs = 600

filePath = "./CKPT/CKP_ep_{0}__loss_{1:.5f}_.h5"
model_path = './CKPT/CKP_ep_576__loss_168.75122_.h5'

loss_fun = custom_mse
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

train_loss = tf.keras.metrics.Mean()
train_kl_loss = tf.keras.metrics.Mean()
valid_loss = tf.keras.metrics.Mean()
sisdr_Metric = SiSdr()
val_sisdr_Metric = SiSdr()

# Model 불러오는 부분이다
vq_vae = Vq_vae(latent_size, gumbel_hard=False)
vq_vae(0, True)
vq_vae.load_weights(model_path)

In [174]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # Call model
        results = vq_vae(x)
        
        loss_value = loss_fun(y, results)
        loss_value += sum(vq_vae.losses) # Add KL loss
    
    # Update weights
    grads = tape.gradient(loss_value, vq_vae.trainable_weights)
    optimizer.apply_gradients(zip(grads, vq_vae.trainable_weights))
    
    # Update loss and si-sdr
    train_loss.update_state(loss_value)
    sisdr_Metric.update_state(y, results)
    
    train_kl_loss.update_state(sum(vq_vae.losses))
    
    return loss_value

@tf.function
def test_step(x, y):
    # Call model
    val_results = vq_vae(x)
    
    val_loss_value = loss_fun(y, val_results)
    val_loss_value += sum(vq_vae.losses) # Add KL loss
    
    # Update loss and si-sdr
    valid_loss.update_state(val_loss_value)
    val_sisdr_Metric.update_state(y, val_results)
    
    return val_loss_value

In [175]:
previous_loss = float('inf')

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))
    start_time = time.time()

    # Iterate over the batches of the dataset
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        x_batch_train = tf.cast(x_batch_train, dtype=tf.float32)
        y_batch_train = tf.cast(y_batch_train, dtype=tf.float32)
        
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 1 batches
#         if step % 1 == 0:
#             print("Training loss (for one batch) at step %d: %.4f" % (step, train_loss.result()))
#             print("Training Si-sdr (for one batch) at step %d: %.4f" % (step, sisdr_Metric.result()))
#             print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Run a validation loop at the end of each epoch
    for x_batch_val, y_batch_val in valid_dataset:
        x_batch_val = tf.cast(x_batch_val, dtype=tf.float32)
        y_batch_val = tf.cast(y_batch_val, dtype=tf.float32)
        
        val_loss_value = test_step(x_batch_val, y_batch_val)
    
    print()
    print('----------------------------------------------------------------------------------')
    print("Time taken >>> %.2fs <<<" % (time.time() - start_time))
    print('epoch: {}, Train_loss: {}, Train_Si-sdr: {}, Train_KL_loss: {} \n\
    Valid_loss: {}, Valid_Si-sdr: {}'.format(
        epoch+1,
        train_loss.result(),
        sisdr_Metric.result(),
        train_kl_loss.result(),
        valid_loss.result(),
        val_sisdr_Metric.result()))
    print('----------------------------------------------------------------------------------')
    
    # Save Model
    if valid_loss.result() < previous_loss:
        filePath_temp = filePath.format(epoch+1, valid_loss.result())
        
        vq_vae.save_weights(filePath_temp)
        print('Epoch {}: val_loss improved from {} to {}, saving model to {}'.format(
            epoch+1,
            previous_loss,
            valid_loss.result(),
            filePath_temp))
        
        previous_loss = valid_loss.result()
    else:
        print('Epoch {}: val_loss did not improve from {}'.format(
            epoch+1,
            previous_loss))
    print()
    
    # Reset metrics at the end of each epoch
    train_loss.reset_states()
    sisdr_Metric.reset_states()
    valid_loss.reset_states()
    val_sisdr_Metric.reset_states()
    
    train_kl_loss.reset_states()
    
    # Data shuffle at the end of each epoch
    train_dataset.on_epoch_end()
    valid_dataset.on_epoch_end()


Start of epoch 1

----------------------------------------------------------------------------------
Time taken >>> 3.19s <<<
epoch: 1, Train_loss: 170.64132690429688, Train_Si-sdr: 2.217930793762207, Train_KL_loss: 58.307884216308594 
    Valid_loss: 165.9063720703125, Valid_Si-sdr: 2.36375093460083
----------------------------------------------------------------------------------
Epoch 1: val_loss improved from inf to 165.9063720703125, saving model to ./CKPT/CKP_ep_1__loss_165.90637_.h5


Start of epoch 2

----------------------------------------------------------------------------------
Time taken >>> 1.01s <<<
epoch: 2, Train_loss: 181.36134338378906, Train_Si-sdr: 1.606610655784607, Train_KL_loss: 58.40456008911133 
    Valid_loss: 176.42575073242188, Valid_Si-sdr: 2.0854380130767822
----------------------------------------------------------------------------------
Epoch 2: val_loss did not improve from 165.9063720703125


Start of epoch 3

--------------------------------------


----------------------------------------------------------------------------------
Time taken >>> 1.02s <<<
epoch: 20, Train_loss: 177.59649658203125, Train_Si-sdr: 2.1253292560577393, Train_KL_loss: 63.61944580078125 
    Valid_loss: 174.72531127929688, Valid_Si-sdr: 2.309779644012451
----------------------------------------------------------------------------------
Epoch 20: val_loss did not improve from 165.9063720703125


Start of epoch 21

----------------------------------------------------------------------------------
Time taken >>> 1.01s <<<
epoch: 21, Train_loss: 171.7360382080078, Train_Si-sdr: 2.530550956726074, Train_KL_loss: 63.045631408691406 
    Valid_loss: 171.48529052734375, Valid_Si-sdr: 2.3708930015563965
----------------------------------------------------------------------------------
Epoch 21: val_loss did not improve from 165.9063720703125


Start of epoch 22

----------------------------------------------------------------------------------
Time taken >>> 1.0


----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 39, Train_loss: 176.87759399414062, Train_Si-sdr: 2.1519222259521484, Train_KL_loss: 63.352088928222656 
    Valid_loss: 173.50804138183594, Valid_Si-sdr: 2.463007926940918
----------------------------------------------------------------------------------
Epoch 39: val_loss did not improve from 165.9063720703125


Start of epoch 40

----------------------------------------------------------------------------------
Time taken >>> 0.99s <<<
epoch: 40, Train_loss: 174.2301025390625, Train_Si-sdr: 2.3745625019073486, Train_KL_loss: 63.930946350097656 
    Valid_loss: 170.62049865722656, Valid_Si-sdr: 2.5669026374816895
----------------------------------------------------------------------------------
Epoch 40: val_loss did not improve from 165.9063720703125


Start of epoch 41

----------------------------------------------------------------------------------
Time taken >>> 0


----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 58, Train_loss: 176.04168701171875, Train_Si-sdr: 1.9879190921783447, Train_KL_loss: 58.84613037109375 
    Valid_loss: 172.21522521972656, Valid_Si-sdr: 2.259948968887329
----------------------------------------------------------------------------------
Epoch 58: val_loss did not improve from 165.9063720703125


Start of epoch 59

----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 59, Train_loss: 172.94662475585938, Train_Si-sdr: 2.2867884635925293, Train_KL_loss: 60.925140380859375 
    Valid_loss: 174.18374633789062, Valid_Si-sdr: 2.3049275875091553
----------------------------------------------------------------------------------
Epoch 59: val_loss did not improve from 165.9063720703125


Start of epoch 60

----------------------------------------------------------------------------------
Time taken >>> 1


----------------------------------------------------------------------------------
Time taken >>> 1.04s <<<
epoch: 77, Train_loss: 173.3389892578125, Train_Si-sdr: 2.182582378387451, Train_KL_loss: 60.04447937011719 
    Valid_loss: 169.20140075683594, Valid_Si-sdr: 2.48701810836792
----------------------------------------------------------------------------------
Epoch 77: val_loss did not improve from 165.9063720703125


Start of epoch 78

----------------------------------------------------------------------------------
Time taken >>> 1.02s <<<
epoch: 78, Train_loss: 171.6219024658203, Train_Si-sdr: 2.372267723083496, Train_KL_loss: 61.03981018066406 
    Valid_loss: 168.11337280273438, Valid_Si-sdr: 2.6053948402404785
----------------------------------------------------------------------------------
Epoch 78: val_loss did not improve from 165.9063720703125


Start of epoch 79

----------------------------------------------------------------------------------
Time taken >>> 1.00s <

...


----------------------------------------------------------------------------------
Time taken >>> 1.01s <<<
epoch: 508, Train_loss: 162.1533203125, Train_Si-sdr: 3.046971321105957, Train_KL_loss: 62.03819274902344 
    Valid_loss: 159.0111846923828, Valid_Si-sdr: 3.200991630554199
----------------------------------------------------------------------------------
Epoch 508: val_loss did not improve from 156.6813201904297


Start of epoch 509

----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 509, Train_loss: 165.22494506835938, Train_Si-sdr: 2.730090618133545, Train_KL_loss: 60.16706848144531 
    Valid_loss: 161.47344970703125, Valid_Si-sdr: 2.8400681018829346
----------------------------------------------------------------------------------
Epoch 509: val_loss did not improve from 156.6813201904297


Start of epoch 510

----------------------------------------------------------------------------------
Time taken >>> 0.9


----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 527, Train_loss: 162.94056701660156, Train_Si-sdr: 2.966662883758545, Train_KL_loss: 61.42108154296875 
    Valid_loss: 160.01065063476562, Valid_Si-sdr: 2.962556838989258
----------------------------------------------------------------------------------
Epoch 527: val_loss did not improve from 156.6080780029297


Start of epoch 528

----------------------------------------------------------------------------------
Time taken >>> 0.99s <<<
epoch: 528, Train_loss: 165.39743041992188, Train_Si-sdr: 2.6423685550689697, Train_KL_loss: 58.92369842529297 
    Valid_loss: 161.3029022216797, Valid_Si-sdr: 2.8261635303497314
----------------------------------------------------------------------------------
Epoch 528: val_loss did not improve from 156.6080780029297


Start of epoch 529

----------------------------------------------------------------------------------
Time taken >>


----------------------------------------------------------------------------------
Time taken >>> 1.01s <<<
epoch: 545, Train_loss: 157.48068237304688, Train_Si-sdr: 3.1532680988311768, Train_KL_loss: 59.5720100402832 
    Valid_loss: 154.5758056640625, Valid_Si-sdr: 3.2737977504730225
----------------------------------------------------------------------------------
Epoch 545: val_loss improved from 154.94232177734375 to 154.5758056640625, saving model to ./CKPT/CKP_ep_545__loss_154.57581_.h5


Start of epoch 546

----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 546, Train_loss: 161.62152099609375, Train_Si-sdr: 2.8255813121795654, Train_KL_loss: 58.09868621826172 
    Valid_loss: 159.3902130126953, Valid_Si-sdr: 2.908630132675171
----------------------------------------------------------------------------------
Epoch 546: val_loss did not improve from 154.5758056640625


Start of epoch 547

---------------------------


----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 564, Train_loss: 164.87786865234375, Train_Si-sdr: 2.535874128341675, Train_KL_loss: 57.082908630371094 
    Valid_loss: 160.48934936523438, Valid_Si-sdr: 2.8276891708374023
----------------------------------------------------------------------------------
Epoch 564: val_loss did not improve from 154.5758056640625


Start of epoch 565

----------------------------------------------------------------------------------
Time taken >>> 1.00s <<<
epoch: 565, Train_loss: 161.5125732421875, Train_Si-sdr: 2.7732274532318115, Train_KL_loss: 57.678611755371094 
    Valid_loss: 157.64572143554688, Valid_Si-sdr: 3.1005406379699707
----------------------------------------------------------------------------------
Epoch 565: val_loss did not improve from 154.5758056640625


Start of epoch 566

----------------------------------------------------------------------------------
Time taken


----------------------------------------------------------------------------------
Time taken >>> 1.02s <<<
epoch: 582, Train_loss: 163.84475708007812, Train_Si-sdr: 2.948310375213623, Train_KL_loss: 61.48612976074219 
    Valid_loss: 159.52378845214844, Valid_Si-sdr: 3.2044599056243896
----------------------------------------------------------------------------------
Epoch 582: val_loss did not improve from 154.3397979736328


Start of epoch 583

----------------------------------------------------------------------------------
Time taken >>> 1.01s <<<
epoch: 583, Train_loss: 164.42433166503906, Train_Si-sdr: 2.775447368621826, Train_KL_loss: 61.317874908447266 
    Valid_loss: 161.92909240722656, Valid_Si-sdr: 2.911527156829834
----------------------------------------------------------------------------------
Epoch 583: val_loss did not improve from 154.3397979736328


Start of epoch 584

----------------------------------------------------------------------------------
Time taken >

# 여기는 기존의 .fit() 함수를 사용해서 학습하는 부분임

In [140]:
tf.random.set_seed(42)

latent_size = 1024
epoch = 300

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_252__loss_45.39530_.h5'
    
    loss_fun = custom_mse
#     loss_fun = custom_sisdr_loss
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=False)

    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    vq_vae.compile(optimizer, loss=loss_fun, metrics=[SiSdr()])
    
    vq_vae(0, True)
    vq_vae.summary()
    
    # 사용 안할 때는 load_model 주석 처리 하자
    vq_vae.load_weights(model_path)
    # ----------------------------------------
    
    tf.executing_eagerly()

history = vq_vae.fit(
    train_dataset,
    epochs=epoch,
    validation_data=valid_dataset,
    shuffle=True,
    callbacks=[checkpoint_cb],
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_30 (Softmax)         (None, None, 1024)        0         
_________________________________________________________________
encoder (Encoder)            (None, None, 1024)        739232    
_________________________________________________________________
embedding_30 (Embedding)     multiple                  0 (unused)
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           1524641   
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 1024)        0         
Total params: 2,263,873
Trainable params: 2,263,873
Non-trainable params: 0
_________________________________________________________________
Epoch 

4/4 [==============================] - 1s 358ms/step - loss: 663.7785 - Si-sdr: 3.5728 - val_loss: 648.5117 - val_Si-sdr: 3.5417

Epoch 00036: val_loss did not improve from 50.88303
Epoch 37/300
4/4 [==============================] - 1s 378ms/step - loss: 639.5051 - Si-sdr: 3.6157 - val_loss: 625.2753 - val_Si-sdr: 3.4231

Epoch 00037: val_loss did not improve from 50.88303
Epoch 38/300
4/4 [==============================] - 1s 371ms/step - loss: 618.8354 - Si-sdr: 3.3118 - val_loss: 604.7725 - val_Si-sdr: 3.2061

Epoch 00038: val_loss did not improve from 50.88303
Epoch 39/300
4/4 [==============================] - 1s 377ms/step - loss: 601.9601 - Si-sdr: 2.9624 - val_loss: 590.2539 - val_Si-sdr: 2.7970

Epoch 00039: val_loss did not improve from 50.88303
Epoch 40/300
4/4 [==============================] - 1s 356ms/step - loss: 582.4575 - Si-sdr: 2.8019 - val_loss: 569.3883 - val_Si-sdr: 2.8109

Epoch 00040: val_loss did not improve from 50.88303
Epoch 41/300
4/4 [====================

4/4 [==============================] - 1s 379ms/step - loss: 258.4825 - Si-sdr: -1.1419 - val_loss: 258.0213 - val_Si-sdr: -1.1217

Epoch 00078: val_loss did not improve from 50.88303
Epoch 79/300
4/4 [==============================] - 1s 366ms/step - loss: 252.8455 - Si-sdr: -0.8841 - val_loss: 251.2665 - val_Si-sdr: -0.8350

Epoch 00079: val_loss did not improve from 50.88303
Epoch 80/300
4/4 [==============================] - 1s 361ms/step - loss: 260.1012 - Si-sdr: -1.4343 - val_loss: 254.9443 - val_Si-sdr: -1.1992

Epoch 00080: val_loss did not improve from 50.88303
Epoch 81/300
4/4 [==============================] - 1s 368ms/step - loss: 254.9350 - Si-sdr: -1.0729 - val_loss: 248.2771 - val_Si-sdr: -0.7258

Epoch 00081: val_loss did not improve from 50.88303
Epoch 82/300
4/4 [==============================] - 1s 361ms/step - loss: 255.0940 - Si-sdr: -1.2344 - val_loss: 252.7626 - val_Si-sdr: -1.1116

Epoch 00082: val_loss did not improve from 50.88303
Epoch 83/300
4/4 [==========


Epoch 00119: val_loss did not improve from 50.88303
Epoch 120/300
4/4 [==============================] - 1s 346ms/step - loss: 241.3756 - Si-sdr: -0.8532 - val_loss: 239.9802 - val_Si-sdr: -0.7279

Epoch 00120: val_loss did not improve from 50.88303
Epoch 121/300
4/4 [==============================] - 1s 363ms/step - loss: 238.3245 - Si-sdr: -0.6810 - val_loss: 233.2504 - val_Si-sdr: -0.4119

Epoch 00121: val_loss did not improve from 50.88303
Epoch 122/300
4/4 [==============================] - 1s 361ms/step - loss: 236.8205 - Si-sdr: -0.7012 - val_loss: 234.0815 - val_Si-sdr: -0.4483

Epoch 00122: val_loss did not improve from 50.88303
Epoch 123/300
4/4 [==============================] - 1s 367ms/step - loss: 238.8212 - Si-sdr: -0.7802 - val_loss: 233.6698 - val_Si-sdr: -0.4835

Epoch 00123: val_loss did not improve from 50.88303
Epoch 124/300
4/4 [==============================] - 1s 357ms/step - loss: 237.3624 - Si-sdr: -0.6211 - val_loss: 233.8785 - val_Si-sdr: -0.4348

Epoch 001

4/4 [==============================] - 1s 362ms/step - loss: 230.6607 - Si-sdr: -0.3119 - val_loss: 231.7780 - val_Si-sdr: -0.3086

Epoch 00161: val_loss did not improve from 50.88303
Epoch 162/300
4/4 [==============================] - 1s 359ms/step - loss: 235.5537 - Si-sdr: -0.5800 - val_loss: 228.3120 - val_Si-sdr: -0.1432

Epoch 00162: val_loss did not improve from 50.88303
Epoch 163/300
4/4 [==============================] - 1s 365ms/step - loss: 234.7994 - Si-sdr: -0.4564 - val_loss: 232.8687 - val_Si-sdr: -0.3446

Epoch 00163: val_loss did not improve from 50.88303
Epoch 164/300
4/4 [==============================] - 1s 368ms/step - loss: 229.4749 - Si-sdr: -0.2582 - val_loss: 227.9099 - val_Si-sdr: -0.1611

Epoch 00164: val_loss did not improve from 50.88303
Epoch 165/300
4/4 [==============================] - 1s 355ms/step - loss: 231.9311 - Si-sdr: -0.4023 - val_loss: 228.9362 - val_Si-sdr: -0.3638

Epoch 00165: val_loss did not improve from 50.88303
Epoch 166/300
4/4 [=====

4/4 [==============================] - 1s 358ms/step - loss: 225.4809 - Si-sdr: -0.1599 - val_loss: 224.6974 - val_Si-sdr: -0.1010

Epoch 00202: val_loss did not improve from 50.88303
Epoch 203/300
4/4 [==============================] - 1s 359ms/step - loss: 223.1396 - Si-sdr: -0.0063 - val_loss: 222.0022 - val_Si-sdr: 0.0047

Epoch 00203: val_loss did not improve from 50.88303
Epoch 204/300
4/4 [==============================] - 1s 359ms/step - loss: 225.4056 - Si-sdr: -0.2021 - val_loss: 219.8166 - val_Si-sdr: 0.0205

Epoch 00204: val_loss did not improve from 50.88303
Epoch 205/300
4/4 [==============================] - 1s 357ms/step - loss: 222.6088 - Si-sdr: -0.0894 - val_loss: 222.5327 - val_Si-sdr: -0.1126

Epoch 00205: val_loss did not improve from 50.88303
Epoch 206/300
4/4 [==============================] - 1s 365ms/step - loss: 221.2665 - Si-sdr: -0.0313 - val_loss: 221.0095 - val_Si-sdr: -0.0107

Epoch 00206: val_loss did not improve from 50.88303
Epoch 207/300
4/4 [=======


Epoch 00243: val_loss did not improve from 50.88303
Epoch 244/300
4/4 [==============================] - 1s 343ms/step - loss: 221.1372 - Si-sdr: 0.2586 - val_loss: 220.1328 - val_Si-sdr: 0.2997

Epoch 00244: val_loss did not improve from 50.88303
Epoch 245/300
4/4 [==============================] - 1s 372ms/step - loss: 221.4229 - Si-sdr: 0.3810 - val_loss: 221.1928 - val_Si-sdr: 0.3218

Epoch 00245: val_loss did not improve from 50.88303
Epoch 246/300
4/4 [==============================] - 1s 368ms/step - loss: 220.5886 - Si-sdr: 0.3085 - val_loss: 220.4405 - val_Si-sdr: 0.2364

Epoch 00246: val_loss did not improve from 50.88303
Epoch 247/300
4/4 [==============================] - 1s 343ms/step - loss: 221.0833 - Si-sdr: 0.2249 - val_loss: 220.4903 - val_Si-sdr: 0.2607

Epoch 00247: val_loss did not improve from 50.88303
Epoch 248/300
4/4 [==============================] - 1s 359ms/step - loss: 223.1148 - Si-sdr: 0.0827 - val_loss: 218.6190 - val_Si-sdr: 0.2606

Epoch 00248: val_lo


Epoch 00285: val_loss did not improve from 50.88303
Epoch 286/300
4/4 [==============================] - 1s 367ms/step - loss: 219.4435 - Si-sdr: 0.3525 - val_loss: 219.3310 - val_Si-sdr: 0.3755

Epoch 00286: val_loss did not improve from 50.88303
Epoch 287/300
4/4 [==============================] - 1s 347ms/step - loss: 217.6745 - Si-sdr: 0.4416 - val_loss: 213.5516 - val_Si-sdr: 0.6151

Epoch 00287: val_loss did not improve from 50.88303
Epoch 288/300
4/4 [==============================] - 1s 341ms/step - loss: 215.5732 - Si-sdr: 0.4514 - val_loss: 215.7825 - val_Si-sdr: 0.4537

Epoch 00288: val_loss did not improve from 50.88303
Epoch 289/300
4/4 [==============================] - 1s 365ms/step - loss: 213.9034 - Si-sdr: 0.6114 - val_loss: 214.5371 - val_Si-sdr: 0.5877

Epoch 00289: val_loss did not improve from 50.88303
Epoch 290/300
4/4 [==============================] - 1s 356ms/step - loss: 214.2224 - Si-sdr: 0.6438 - val_loss: 215.0473 - val_Si-sdr: 0.4920

Epoch 00290: val_lo

## 2.2. Encoder 부르는 방법, Decoder에 값 넣는 방법

In [50]:
latent_size = 512
epoch = 200
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_283__loss_141.77045_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=False)
    vq_vae(0, True)
    vq_vae.summary()
    
    vq_vae.load_weights(model_path)
    
    # 이렇게 하면, transforer의 input으로 들어가는 one-hot 형식의 값을 얻을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        encode_onehot = tf.cast(tf.equal(encode, tf.math.reduce_max(encode, 2, keepdims=True)), encode.dtype)
    
    # 이렇게 하면, transformer의 output을 vq-vae의 decoder 입력으로 넣을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        encode_onehot = tf.cast(tf.equal(encode, tf.math.reduce_max(encode, 2, keepdims=True)), encode.dtype)
        
        # 이렇게 이전 layer의 출렫을 넣으면 됨
        decode = vq_vae.decoder(encode_onehot).numpy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_25 (Softmax)         (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 3. Test Model

In [110]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [111]:
mkdir_p('./test_wav/') # Result wav 폴더 만드는 코드

In [112]:
def audiowrite(data, path, samplerate=16000, normalize=False, threaded=True):
    """ Write the audio data ``data`` to the wav file ``path``
    The file can be written in a threaded mode. In this case, the writing
    process will be started at a separate thread. Consequently, the file will
    not be written when this function exits.
    :param data: A numpy array with the audio data
    :param path: The wav file the data should be written to
    :param samplerate: Samplerate of the audio data
    :param normalize: Normalize the audio first so that the values are within
        the range of [INTMIN, INTMAX]. E.g. no clipping occurs
    :param threaded: If true, the write process will be started as a separate
        thread
    :return: The number of clipped samples
    """
    data = data.copy()
    int16_max = np.iinfo(np.int16).max
    int16_min = np.iinfo(np.int16).min

    if normalize:
        if not data.dtype.kind == 'f':
            data = data.astype(np.float)
        data /= np.max(np.abs(data))

    if data.dtype.kind == 'f':
        data *= int16_max

    sample_to_clip = np.sum(data > int16_max)
    if sample_to_clip > 0:
        print('Warning, clipping {} samples'.format(sample_to_clip))
    data = np.clip(data, int16_min, int16_max)
    data = data.astype(np.int16)

    if threaded:
        threading.Thread(target=wav_write, args=(path, samplerate, data)).start()
    else:
        wav_write(path, samplerate, data)

    return sample_to_clip

In [115]:
with tf.device('/cpu:0'):
    latent_size = 1024
    sample_rate = 8000
    model_path = './CKPT/CKP_ep_299__loss_70.91695_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=True)
    vq_vae(0, True)
    vq_vae.summary()
    vq_vae.load_weights(model_path)

    for batch in test_dataset:
        input_batch, length_batch, name = batch

        result = vq_vae.predict(input_batch)
        
        wav_name = './test_wav/' + name[0][:-5] + '_s1.wav'
        audiowrite(result[0], wav_name, sample_rate, True, True)

Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_20 (Softmax)         (None, None, 1024)        0         
_________________________________________________________________
encoder (Encoder)            (None, None, 1024)        739232    
_________________________________________________________________
embedding_20 (Embedding)     multiple                  0 (unused)
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           1524641   
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 1024)        0         
Total params: 2,263,873
Trainable params: 2,263,873
Non-trainable params: 0
_________________________________________________________________


# 여기 밑에는 연습장임

In [63]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(None, 1)))
model.add(layers.Conv1D(filters=512, kernel_size=3, padding='same'))

input_array = np.random.randn(2, 10, 1)
input_array2 = np.random.randn(2, 9, 1)
with tf.device('/cpu:0'):
    model.compile('rmsprop', 'mse')

    output_array = model.predict(input_array)
    output_array2 = model.predict(input_array2)

In [64]:
tf.reduce_sum(output_array, axis=[1, 2])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-3.2364433, -5.300486 ], dtype=float32)>

In [65]:
print(output_array)
print(output_array.shape)
print(output_array2.shape)

[[[ 0.00213541  0.01945671 -0.01550578 ... -0.00601448  0.01089236
    0.03509172]
  [-0.00979825 -0.01161936 -0.03782888 ... -0.01964907  0.02604836
   -0.01746842]
  [-0.01680236 -0.04395067  0.00300831 ...  0.02191263  0.02651824
   -0.0194194 ]
  ...
  [ 0.03252641 -0.03450746 -0.01116706 ...  0.04157236  0.00767434
    0.00222839]
  [ 0.03689077 -0.00653183  0.0138604  ...  0.03399033  0.01157293
   -0.01727439]
  [ 0.00956812  0.00808328 -0.00985544 ...  0.02565406  0.01499317
    0.0055492 ]]

 [[ 0.00062903 -0.02250458 -0.01209254 ... -0.00480657  0.02518445
    0.00395943]
  [ 0.00067916 -0.03764667 -0.00435403 ... -0.00190964  0.0198026
   -0.01638822]
  [ 0.03713017 -0.02867446 -0.01009638 ...  0.03275302  0.02717083
    0.00666822]
  ...
  [ 0.01911856  0.02970011 -0.00635595 ... -0.01212452  0.03101327
   -0.01940764]
  [-0.04032427 -0.04538781 -0.00670558 ...  0.00985033  0.0156134
   -0.02115321]
  [ 0.0188317  -0.04442241  0.00422117 ...  0.03525497 -0.0081987
   -0.036

In [69]:
one_hot = tf.math.argmax(tf.cast(tf.equal(output_array, tf.math.reduce_max(output_array, 2, keepdims=True)), output_array.dtype), axis=-1)
print(one_hot.shape)
print(one_hot)
# layers.Embedding(512, 512)(one_hot)

(2, 10)
tf.Tensor(
[[228 249 283 206  20 206 435  32 270  30]
 [428 206  20 244 357 289 324 249 498 134]], shape=(2, 10), dtype=int64)


In [70]:
one_hot = tf.math.argmax(output_array, axis=-1)
print(one_hot.shape)
print(one_hot)

(2, 10)
tf.Tensor(
[[228 249 283 206  20 206 435  32 270  30]
 [428 206  20 244 357 289 324 249 498 134]], shape=(2, 10), dtype=int64)


In [50]:
qy = tf.nn.softmax(output_array)
log_qy = tf.math.log(qy + 1e-10)
log_uniform = qy * (log_qy - tf.math.log(1.0 / 512))
kl_loss = tf.reduce_sum(log_uniform, axis=[1, 2])
kl_loss = tf.reduce_mean(kl_loss)

print(kl_loss)

tf.Tensor(0.012074914, shape=(), dtype=float32)


In [30]:
batch_size = tf.shape(output_array)[0]
array1_size = tf.shape(output_array)[1]
array2_size = tf.shape(output_array2)[1]
feature_size = tf.shape(output_array)[-1]

if array1_size < array2_size:
#     append_size = array1_size - array2_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array2 = tf.concat([output_array2, append_zeros], axis=1)
    output_array2 = tf.slice(output_array2, [0, 0, 0], [-1, array1_size, -1])
elif array1_size > array2_size:
#     append_size = array2_size - array1_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array = tf.concat([output_array, append_zeros], axis=1)
    output_array = tf.slice(output_array, [0, 0, 0], [-1, array2_size, -1])

print(output_array.shape)
print(output_array2.shape)
# output_array0 = output_array[1]
# output_array20 = output_array2[1]
# target = np.sum(output_array20 * output_array0) * output_array0 / np.square(np.linalg.norm(output_array0, ord=2))
# noise = output_array20 - target
# npnp = 10 * np.log10(np.square(np.linalg.norm(target, ord=2)) / np.square(np.linalg.norm(noise, ord=2)))
# print(npnp)

target = tf.linalg.matmul(output_array2, output_array, transpose_a=True) * output_array / tf.expand_dims(tf.experimental.numpy.square(tf.norm(output_array, axis=1)), axis=-1)
noise = output_array2 - target
si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
si_sdr = tf.reduce_mean(si_sdr)
print(si_sdr)

(2, 9, 1)
(2, 9, 1)
tf.Tensor(2.8309882, shape=(), dtype=float32)


In [20]:
tf.cast(tf.equal(output_array, tf.math.reduce_max(output_array, 2, keepdims=True)), output_array.dtype)

<tf.Tensor: shape=(2, 9, 4), dtype=float32, numpy=
array([[[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)>

In [21]:
output_array

<tf.Tensor: shape=(2, 9, 4), dtype=float32, numpy=
array([[[-0.03009652, -0.03612775, -0.06680483, -0.03670201],
        [-0.04768711, -0.12344762, -0.03924457, -0.11762322],
        [ 0.01808495, -0.16106637, -0.19467078, -0.15282159],
        [-0.0986427 , -0.08625205, -0.12661007, -0.16366175],
        [-0.09758376, -0.08886974, -0.0433558 , -0.19985165],
        [-0.06933096, -0.03154394, -0.13725929, -0.20143284],
        [ 0.03375649,  0.00182091, -0.01022564, -0.35924646],
        [-0.01645333, -0.10466891, -0.13975918, -0.12066491],
        [-0.13588801, -0.08173112, -0.00253745, -0.28615874]],

       [[ 0.04865369, -0.02880372, -0.06414615, -0.07730438],
        [-0.08225074, -0.03192509, -0.06216412, -0.08035193],
        [-0.09515338,  0.04221668,  0.14230826, -0.23082384],
        [-0.00094383,  0.05597762, -0.09290768, -0.08630683],
        [-0.09894791, -0.04727853, -0.01004983, -0.30325216],
        [ 0.01705559, -0.16948727, -0.08829505, -0.16453639],
        [-0.07230

In [67]:
output_softmax = tf.nn.softmax(output_array)
output_softmax

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [73]:
output_reshape = tf.reshape(output_softmax, [-1, 4])
output_reshape.shape

TensorShape([10, 4])

In [83]:
tf.reshape(tf.nn.softmax(output_array), [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [76]:
# tf.cast(tf.equal(y, tf.reduce_max(y,1,keep_dims=True)), y.dtype)
output_hard = tf.cast(tf.equal(output_reshape, tf.math.reduce_max(output_reshape, 1, keepdims=True)), output_softmax.dtype)
output_hard

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>

In [81]:
tf.reshape(output_hard, [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]], dtype=float32)>

In [322]:
tf.transpose(output_array, perm=[0, 2, 1]).shape

TensorShape([2, 4, 5])

In [316]:
layers.Softmax(output_array)

In [317]:
np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]).shape

(3, 3)

In [65]:
np.log(10)

2.302585092994046